# Value iteration
This is the value iteration solution for UAV environment

In [ ]:
import numpy as np
import torch
import copy
from utils import *
import torch
import torch.nn as nn
from random import randint, choice
from IPython.display import clear_output
import matplotlib.pyplot as plt
import math
import random

In [ ]:
class UAVEnvironment():
    """
    Game environment for UAV test
    
    ---Map---
    
    y-axis(length)
    |
    |
    |
    |
    |
    |
     _______________________ x-axis(width)
     
    Hight is a fixed value
    
    """
    def __init__(self, config):
        # Game config
        self.action_space = (0, 1, 2, 3) # up, right, down, left, total 4 actions
        self.total_steps = config["total_steps"] # when the game end
        self.current_step = 0
        if config["is_random_env"] == False:
            self.random_seed = config["random_seed"]
            random.seed(self.random_seed)
        
        # Map config
        self.map = dict(width=config["map"]["width"], length=config["map"]["length"], height=config["map"]["height"])
        self.UAV_speed = config["UAV_speed"]
        self.UAV_initial_pos = config["UAV_initial_pos"] # a tuple
        self.UAV_current_pos = self.UAV_initial_pos
        self.number_of_user = config["number_of_user"]
        self.users_pos = list()
        self.UAV_path = [] # record the path of UAV
        for i in range(0, self.number_of_user):
            self.users_pos.append((randint(0, self.map["width"]), randint(0, self.map["length"])))
        
        # Wireless config
        self.g0 = config["wireless_parameter"]["g0"]
        self.B = config["wireless_parameter"]["B"]
        self.Pk = config["wireless_parameter"]["Pk"]
        self.noise = config["wireless_parameter"]["noise"]
        
    def get_reward(self, UAV_pos):
        # One step Reward is define as the summation of all user's utility
        reward = 0
        for user_index in range(0, self.number_of_user):
            gkm = self.g0 / (self.map["height"] ** 2 + (UAV_pos[0] - self.users_pos[user_index][0]) ** 2 + (UAV_pos[1] - self.users_pos[user_index][1]) ** 2)
            #user_utility = self.B * math.log(1 + self.Pk * gkm / self.noise, 2)
            user_utility = (self.B/self.number_of_user)* math.log(1 + self.Pk * gkm / self.noise, 2)
            reward = reward + user_utility
        return reward / (10 ** 6) # Use Mkbps as signal basic unit
    
    def transition_dynamics(self, action, speed, state):
        # given the action (direction), calculate the next state (UAV current position)
        assert action in self.action_space
        next_UAV_pos = list(state)
        if action == 0:
            # move up
            next_UAV_pos[1] = min(next_UAV_pos[1] + speed, self.map["length"])
        if action == 1:
            # move right
            next_UAV_pos[0] = min(next_UAV_pos[0] + speed, self.map["width"])
        if action == 2:
            # move down
            next_UAV_pos[1] = max(next_UAV_pos[1] - speed, 0)
        if action == 3:
            # move left
            next_UAV_pos[0] = max(next_UAV_pos[0] - speed, 0)
        return tuple(next_UAV_pos)
    
    def get_transition(self):
        # This function only works for model based, we are trying to disable this function to try more algorithm
        # Return a table of transition, we assume UAV use fixed flying speed
        """
        Structure:
        transition[
            x_0[
                y_0[
                    {next_state, reward}, # for action 1
                    {next_state, reward}, # for action 2
                    ...
                    {next_state, reward}, # for action 20
                ],
                y_1*v[],
                ...
                y_h-1*v[]
            ],
            x_1*v[],
            x_2*v[],
            ...
            x_w-1*v[]
        ]
        """
        transition = list()
        for state_x in range(0, int(self.map["width"] / self.UAV_speed) + 1):
            transition.append(list())
            for state_y in range(0, int(self.map["length"] / self.UAV_speed) + 1):
                transition[state_x].append(list())
                for action in self.action_space:
                    next_state = self.transition_dynamics(action, self.UAV_speed, (state_x * self.UAV_speed, state_y * self.UAV_speed))
                    reward = self.get_reward(next_state)
                    transition[state_x][state_y].append(dict(next_state=next_state,reward=reward))
        return transition
                    
    def step(self, action, speed=-1):
        # assume we use the max speed as the default speed, when come near to the opt-position, we can slow down the speed
        if speed < 0 or speed >= self.UAV_speed:
            speed = self.UAV_speed
            
        self.UAV_path.append(self.UAV_current_pos)
        self.UAV_current_pos = self.transition_dynamics(action, speed, self.UAV_current_pos)
        self.current_step = self.current_step + 1
        done = False
        if self.current_step == self.total_steps:
            done =  True
        return self.UAV_current_pos, self.get_reward(self.UAV_current_pos), done
    
    def action_sample(self):
        return choice(self.action_space)
    
    def reset(self):
        self.current_step = 0
        self.UAV_current_pos = self.UAV_initial_pos
        return self.UAV_current_pos
        
    def print_attribute(self):
        attrs = vars(self)
        print(', '.join("%s: %s" % item for item in attrs.items()))
        
    def print_locations(self):
        print("UAV position is: {}".format(self.UAV_current_pos))
        print("Users position are: {}".format(self.users_pos))
        
    def print_map(self):
        x_list = [pos[0] for pos in self.users_pos]
        y_list = [pos[1] for pos in self.users_pos]
        x_list.append(self.UAV_current_pos[0])
        y_list.append(self.UAV_current_pos[1])
        for i in range(0, len(self.UAV_path)):
            x_list.append(self.UAV_path[i][0])
            y_list.append(self.UAV_path[i][1])
        
        colors = np.array(["red", "green", "blue"])
        sizes = []
        colors_map = []
        for i in range(0, self.number_of_user):
            sizes.append(25)
            colors_map.append(1)
        sizes.append(50)
        colors_map.append(0)
        for i in range(0, len(self.UAV_path)):
            sizes.append(10)
            colors_map.append(2)
        for i in range(0, len(self.UAV_path) - 1):
            x_values = [self.UAV_path[i][0], self.UAV_path[i+1][0]]
            y_values = [self.UAV_path[i][1], self.UAV_path[i+1][1]]
            plt.plot(x_values, y_values, 'b')
        plt.scatter(x_list, y_list, c=colors[colors_map], s=sizes) 
        plt.axis([0, self.map["width"], 0, self.map["length"]])
        plt.show()
        

In [ ]:
def evaluate(policy, config, num_episodes=1, render=False):
    env = UAVEnvironment(config)
    
    rewards = []
    for i in range(num_episodes):
        obs = env.reset()
        # all policy will return a direction and a speed
        act_direction, act_speed = policy(obs)
        ep_reward = 0
        while True:
            obs, reward, done = env.step(act_direction, act_speed)
            act_direction, act_speed = policy(obs)
            ep_reward += reward
            if done:
                break
            if render == True:
                clear_output(wait=True)
                env.print_attribute()
                print("Current Step: {}".format(env.current_step))
                print("Policy choice direction: {}, speed: {}".format(act_direction, act_speed))
                print("UAV current position x: {}, y: {}".format(env.UAV_current_pos[0], env.UAV_current_pos[1]))
                print("Current step reward: {}, episodes rewards: {}".format(reward, ep_reward))
                env.print_map()
                wait(sleep=0.2)
        rewards.append(ep_reward)
    return np.mean(rewards)

def run(trainer_cls, config=None, reward_threshold=None):
    pass


In [ ]:
# Basic Class for all RL algorithm
class UAVTrainer: 
    def __init__(self, config):
        self.config = config
        self.env = UAVEnvironment(self.config)

# Value Iteration, Tabular, transition dynamic is known, assume only use fixed speed to reduce action space
class UAVTrainerValueIteration(UAVTrainer):
    def __init__(self, config):
        super().__init__(config)
        self.config = config
        self.transitions = self.env.get_transition()
        self.q_table = []
        self.obs_dim = (int(self.env.map["width"] / self.env.UAV_speed), int(self.env.map["length"] / self.env.UAV_speed))
        self.act_dim = len(self.env.action_space)
        self.gamma = config["gamma"]
        
        for x in range(0, self.obs_dim[0]+1):
            self.q_table.append(list())
            for y in range(0, self.obs_dim[1]+1):
                self.q_table[x].append(0)
            
    def get_transition(self, state, act):
        transition = self.transitions[state[0]][state[1]][act]
        return transition["next_state"], transition["reward"]
    
    def print_transitions(self):
        print("Transition width {}, length {}, number of act {}".format(len(self.transitions), len(self.transitions[0]), len(self.transitions[0][0])))
        print(self.transitions)
        
    def print_table(self):
        for j in range(len(self.q_table[0])-1, -1, -1):
            for i in range(0, len(self.q_table)):
                print(self.q_table[i][j], end =" ")
            print("")
            
            
    def copy_current_table(self):
        old_table = []
        for x in range(0, self.obs_dim[0]+1):
            old_table.append(list())
            for y in range(0, self.obs_dim[1]+1):
                old_table[x].append(self.q_table[x][y])
        return old_table

    def update_value_function(self):
        old_table = self.copy_current_table()
        for state_x in range(self.obs_dim[0] + 1):
            for state_y in range(self.obs_dim[1] + 1):
                state_value = 0
                state_action_values = [0 for i in range(0, self.act_dim)]

                for act in range(self.act_dim):
                    next_state, reward = self.get_transition((state_x, state_y), act)
                    table_x = int(next_state[0] / self.env.UAV_speed)
                    table_y = int(next_state[1] / self.env.UAV_speed)
                    #print(table_x, table_y)
                    state_action_values[act] = state_action_values[act] + reward + self.gamma * old_table[table_x][table_y]   
                state_value = np.max(state_action_values)
                self.q_table[state_x][state_y] = state_value
                #print("Update x: {}, y: {} to value {}".format(state_x, state_y, state_value))
            
    def train(self):
        old_state_value_table = self.copy_current_table()
        current_step = 0
        while current_step < self.config['max_iteration']:  
            current_step = current_step + 1
            self.update_value_function()
            if current_step % self.config["evaluate_interval"] == 0:
                print("Iteration {}, Mean Reward is: {}".format(current_step, evaluate(self.policy, config = self.config, num_episodes=1, render=False)))
                #print("Iteration {}, Mean Reward is: {}".format(current_step, 0))
                # check exist
                stop = True
                flag = 0
                for x in range(self.obs_dim[0] + 1):
                    for y in range(self.obs_dim[1] + 1):
                        if abs(self.q_table[x][y] - old_state_value_table[x][y]) > self.config["return_threshold"]:
                            stop = False
                            flag = 1
                    if flag == 1:
                        break
                if stop == True:
                    print("Train converge at i = {}".format(current_step))
                    current_step = self.config['max_iteration']
                else:
                    old_state_value_table = self.copy_current_table()

    def policy(self, obs):
        table_x = int(obs[0] / self.env.UAV_speed)
        table_y = int(obs[1] / self.env.UAV_speed)
        next_state_value_list = []
        for act in range(0, self.act_dim):
            next_state, reward = self.get_transition((table_x, table_y), act)
            next_state_x = int(next_state[0] / self.env.UAV_speed)
            next_state_y = int(next_state[1] / self.env.UAV_speed)
            next_state_value_list.append(self.q_table[next_state_x][next_state_y])
        act = np.argmax(next_state_value_list)
        return act, self.env.UAV_speed

In [ ]:
environment_config = dict(
    total_steps = 50,
    random_seed = 10,
    is_random_env = False,
    map=dict(
        width=1000,
        length=1000,
        height=300
    ),
    number_of_user = 10,
    UAV_speed = 50,
    UAV_initial_pos = (0, 0),
    wireless_parameter = dict(
        g0 = 10 ** (-5),
        B = 10 ** (6),
        Pk = 0.1,
        noise = 10 ** (-9)
    )
)

value_iteration_config = merge_config(dict(
    max_iteration=10000,
    evaluate_interval=100,  # don't need to update policy each iteration
    gamma=0.9,
    return_threshold=1,
), environment_config)
trainer = UAVTrainerValueIteration(value_iteration_config)
trainer.env.print_locations()
trainer.train()

print("Mean Reward is: {}".format(evaluate(trainer.policy, config = value_iteration_config, num_episodes=1, render=True))) # Enable render=True can see the agent behavior

In [ ]:
# The rest is we try to create more complex environment, with more action space and state, not finished yet, please ignore the rest

In [ ]:
import math 

class UAVEnvironmentComplex(UAVEnvironment):
    """
    Complex UAV environment, action can be compose as (speed, direction), UAV position can be continous float number
    
    State = (self.UAV_current_pos, self.users_pos [list])
    Action = (speed, direction)
    
    """
    def __init__(self, config):
        super(UAVEnvironmentComplex, self).__init__(config)
        delattr(self, "action_space") 


    def transition_dynamics(self, action, state):
        # action = (speed, direction), speed with 0 and self.UAV_speed, direction with (0, 2 * pi)
        speed = action[0]
        direction = action[1]
        next_x = self.UAV_current_pos[0] + speed * math.cos(direction)
        next_y = self.UAV_current_pos[1] + speed * math.sin(direction)
        next_x = max(0, next_x)
        next_x = min(self.map["width"], next_x)
        next_y = max(0, next_y)
        next_y = min(self.map["length"], next_y)
        return (next_x, next_y)
  

    def reset(self):
        self.current_step = 0
        self.UAV_current_pos = self.UAV_initial_pos
        return (self.UAV_current_pos, self.users_pos)
    
    def step(self, action):
        # action = (speed, direction)
        # This function return the state = (self.UAV_current_pos, self.users_pos [list])
        speed = action[0]
        speed = max(0, speed)
        speed = min(self.UAV_speed, speed)
        
        standarded_action = (speed, action[1])
        self.UAV_current_pos = self.transition_dynamics(standarded_action, self.UAV_current_pos)
        self.current_step = self.current_step + 1
        done = False
        if self.current_step == self.total_steps:
            done =  True
        state = (self.UAV_current_pos, self.users_pos)
        reward = self.get_reward(self.UAV_current_pos)
        return state, reward, done
    
    def action_sample(self):
        speed = random.uniform(0, 1) * self.UAV_speed
        random_direction = math.pi * 2 * random.uniform(0, 1)
        action = (speed, random_direction)
        return action

In [ ]:
myGame = UAVEnvironmentComplex(environment_config)
myGame.print_locations()
myGame.print_map()

In [ ]:
def evaluate_complex(policy, config, num_episodes=1, render=False):
    env = UAVEnvironmentComplex(config)
    rewards = []
    for i in range(num_episodes):
        obs = env.reset()
        # all policy will return a direction and a speed
        action = policy(obs)
        ep_reward = 0
        while True:
            obs, reward, done = env.step(action)
            action = policy(obs)
            ep_reward += reward
            if done:
                break
            if render == True:
                clear_output(wait=True)
                env.print_locations()
                print("Current Step: {}".format(env.current_step))
                print("Policy choice direction: {}, speed: {}".format((action[1] * 180 / math.pi), action[0]))
                print("Current step reward: {}, episodes rewards: {}".format(reward, ep_reward))
                env.print_map()
                wait(sleep=0.2)
        rewards.append(ep_reward)
    return np.mean(rewards)

In [ ]:
# Basic Class for all RL algorithm
class UAVComplexTrainer: 
    def __init__(self, config):
        self.config = config
        self.env = UAVEnvironmentComplex(self.config)
            
    def train(self):
        pass
    
    def compute_values(self, state):
        pass
    
    def state_to_feature_vector(self, UAV_pos, User_pos_list):
        # State (UAV_pos, Users_pos), assmue there is max_number_of_user which UAV can be server, to define the feature vector length
        # Feature vector: a vector contain tuple of positions
        capacity = self.config["max_number_of_user"] + 1 # UAV + user
        features = [0] * capacity * 2
        features[0] = UAV_pos[0]
        features[1] = UAV_pos[1]
        for i in range(0, len(User_pos_list)):
            features[2*i+2] = User_pos_list[i][0]
            features[2*i+3] = User_pos_list[i][1]
        return features

    def policy(self):
        pass

In [ ]:
# Start from random policy
class UAVComplexTrainerRandomPolicy(UAVComplexTrainer):
    def __init__(self, config):
        super(UAVComplexTrainerRandomPolicy, self).__init__(config)
        
    def policy(self, obs):
        action = self.env.action_sample()
        return action

random_policy_config = environment_config
trainer = UAVComplexTrainerRandomPolicy(random_policy_config)
print("Mean Reward is: {}".format(evaluate_complex(trainer.policy, config = random_policy_config, num_episodes=1, render=True))) # Enable render=True can see the agent behavior

In [ ]:
import torch
from torch import nn

def to_tensor(x):
    """A helper function to transform a numpy array to a Pytorch Tensor"""
    if isinstance(x, list):
        x = np.array(x) 
    if isinstance(x, np.ndarray):
        x = torch.from_numpy(x).type(torch.float32)
    assert isinstance(x, torch.Tensor)
    return x


class NetworkModel(nn.Module):
    def __init__(self, obs_dim, act_dim):
        super(NetworkModel, self).__init__()
        self.network = torch.nn.Sequential(
            torch.nn.Linear(obs_dim,100),
            torch.nn.ReLU(),
            torch.nn.Linear(100,100),
            torch.nn.ReLU(),
            torch.nn.Linear(100,act_dim),
        )

    def forward(self, obs):
        return self.network(obs)

In [ ]:
# Basic Class for all RL algorithm
class UAVComplexTrainerNN(UAVComplexTrainer): 
    def __init__(self, config):
        super(UAVComplexTrainerNN, self).__init__(config)
        self.max_episode_length = self.config["max_episode_length"]
        self.learning_rate = self.config["learning_rate"]
        self.gamma = self.config["gamma"]
        self.model = NetworkModel((self.config["max_number_of_user"] + 1) * 2, 2)

    def train(self):
        pass
    
    def policy(self, state):
        if np.random.uniform(0,1) <= self.config["eps"]:
            action = self.env.action_sample()
        else:
            feature = self.state_to_feature_vector(state[0], state[1])
            model_input = to_tensor(feature).squeeze()
            action = self.model(model_input)
        return action

In [ ]:
linear_function_config = merge_config(dict(
    total_steps = 50,
    number_of_user = 10,
    max_number_of_user = 20,
    map=dict(
        width=1000,
        length=1000,
        height=100
    ),
    max_episode_length=10000,
    eps=0.01,
    gamma=0.9,
    parameter_std=0.01,
    learning_rate=0.01,
), environment_config)

NNTrainer = UAVComplexTrainerNN(linear_function_config)
print("Mean Reward is: {}".format(evaluate_complex(NNTrainer.policy, config = linear_function_config, num_episodes=1, render=True))) # Enable render=True can see the agent behavior

In [ ]:
mu, sigma = 0, 0.1 # mean and standard deviation
s = np.random.normal(mu, sigma, (10,3))
print(s)